# Faithfulness / Mismatch between integration methods

Suppose to have two waveforms, generated by the same mechanism, but with some slight differences in the process. In our case, we want to compare the waveform generated by the trapezoidal rule and the ODE integration method. This will shape how the analysis will go from this point on, because if the ODE is different, this means that we must find a way to make it faster or change the sampling method for the posterior. 

The mismatch can be estimated by projecting one waveform function onto the other in the functions space: this is done through the $\textbf{inner product}$: 

$$(h_i|h_j) = \int_f^{f_{\mathrm{isco}}} \frac{4 h_i^*(f) \cdot h_j(f)}{S_n} \mathrm{d}f \text{,}$$ 

where the two waveforms are inficated with $h$ and $S_n$ indicates the noise spectrum of the instrument. 
This projection needs to be normalized to better have an extimate such that:

$$(h_i|h_j) \in [0, 1] \text{,}$$

with $0$ being total mismatch, and $1$ being almost identical. To do so, we need to evaluate their norm singularly: we take the inner product of the functions with themselves.

$$|h_i|^2 = \int_f^{f_{\mathrm{isco}}} \frac{4 h_i^*(f) \cdot h_i(f)}{S_n} \mathrm{d}f\text{,}$$

$$|h_j|^2 = \int_f^{f_{\mathrm{isco}}} \frac{4 h_j^*(f) \cdot h_j(f)}{S_n} \mathrm{d}f\text{.}$$

Finally, we can evaluate the normalized projection as: 

$$P_n = \frac{(h_i|h_j)}{\sqrt{|h_i|^2 \cdot |h_j|^2}}$$

The mismatch, the real mismatch, is given by the difference between $1$ (perfect superposition), and the result from the above equations: 

$$\mathcal{M} = 1 - P_n\text{.}$$

### Note: 

The functions that are reported are the ones from the $\texttt{environments_handy_functions.py}$ library. I used them here for simplicity pourposes. 

### Import from libraries

In [90]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import scipy as sp
from typing import Callable, NamedTuple, Tuple, Type, Union
from scipy.integrate import quad_vec, simpson
from scipy.special import hyp2f1, betainc
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from matplotlib.colors import LogNorm

from scipy.interpolate import griddata
from scipy.stats import scoreatpercentile
from scipy.integrate import cumulative_trapezoid, cumulative_simpson
from scipy.interpolate import CubicSpline
from scipy.integrate import quad, solve_ivp, simps, cumulative_trapezoid
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar

from pydd.analysis import get_match_pads
from pydd.binary import get_m_1, get_f_isco, get_m_2

import dynesty
from dynesty import plotting as dyplot

In [91]:
import environments_handy_functions

# Reload libraries (in case you change something)
import importlib
importlib.reload(environments_handy_functions)

from environments_handy_functions import (
    df_dt,  
    #find_grid, 
    #f_1yr, 
    h_0, 
    #mycalculate_SNR, 
    #amplitude, 
    #Psi,
    myVacuumBinary, myAccretionDisk, myDarkMatter, myCombination)

Set constants and plot preferences.

In [92]:
G = 6.67408e-11  # m^3 s^-2 kg^-1
C = 299792458.0  # m/s
MSUN = 1.98855e30  # kg
PC = 3.08567758149137e16  # m
YR = 365.25 * 24 * 3600  # s

In [93]:
plt.rcParams['agg.path.chunksize'] = 1000  # Set a larger value for chunksize
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.sans-serif": ["TimesNewRoman"]})
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["axes.labelsize"] = "14"

### Select noise pattern

In [94]:
# Set detector
detector = "LISA"

In [95]:
# Set PSDs, choose observation time and SNR threshold (will set distance in signal system below)
if detector == "et":
    from pydd.noise import S_n_et as S_n, f_range_et as f_range_n  # ET

    T_OBS = 1 * YR
    SNR_THRESH = 12.0
    TITLE = "Einstein Telescope"
elif detector == "ce":
    from pydd.noise import S_n_ce as S_n, f_range_ce as f_range_n  # CE

    T_OBS = 1 * YR
    SNR_THRESH = 12.0
    TITLE = "Cosmic Explorer"
elif detector == "aLIGO":
    from pydd.noise import S_n_aLIGO as S_n, f_range_aLIGO as f_range_n  # aLIGO

    T_OBS = 1 * YR 
    SNR_THRESH = 12.0
    TITLE = "aLIGO"
elif detector == "LISA":
    from pydd.noise import S_n_LISA as S_n, f_range_LISA as f_range_n  # LISA

    T_OBS = 1 * YR #seconds
    SNR_THRESH = 100.0
    TITLE = "LISA"

# Selected functions

### $\cdot$ Phase to coalescence

In [96]:
def phase_f_cumul(f_grid, params):
    '''Phase to coalescence integrated using simple cumulative rule (faster!).'''
    to_integrate = 2 * np.pi * df_dt(f_grid, params)**(-1) * f_grid
    phase = cumulative_trapezoid(to_integrate, f_grid, initial=0)
    return phase
    
def phase_f(f_grid, params):
    '''Finds the binary phase as a function of frequency.'''
    def to_integrate(f):
        function = 2 * np.pi * df_dt(f, params)**(-1) * f
        return interp1d(f, function) # , kind='cubic', fill_value="extrapolate"
    to_integrate_function = to_integrate(f_grid)
    # Differential equation for the phase
    def phase_ode(f, y):
        return to_integrate_function(f)
    # Initial condition
    y0 = [0]
    # Solving the IVP
    result = solve_ivp(phase_ode, [f_grid[0], f_grid[-1]], y0, t_eval=f_grid, rtol=1e-13, atol=1e-13)
    # Extracting the solution
    phase_f = result.y[0]
    return phase_f

### $\cdot$ Time to coalescence

In [97]:
def time_to_coal(f_grid, params):
    '''Time to coalescence integrated using ODE rule.'''
    def to_integrate(f):
        function = df_dt(f, params)**(-1)
        return interp1d(f, function) # , kind='cubic', fill_value="extrapolate"
    to_integrate_function = to_integrate(f_grid)
    # Differential equation for the time to coalescence
    def time_to_coal_int(f, y):
        return to_integrate_function(f)
    # Initial condition
    y0 = [0]
    # Solving the IVP
    result = solve_ivp(time_to_coal_int, [f_grid[0], f_grid[-1]], y0, t_eval=f_grid, rtol=1e-13, atol=1e-13)
    # Extracting the solution
    t_coal_f = result.y[0]
    return t_coal_f

def time_to_coal_cumul(f_grid, params):
    '''Time to coalescence integrated using simple cumulative rule (faster!).'''
    to_integrate = df_dt(f_grid, params)**(-1)
    time = cumulative_trapezoid(to_integrate, f_grid, initial=0)
    return time

### $\cdot$ $\Phi_T$ and $\Psi$

In [107]:
# ODE method --------------------------------------------------------------------

def PhiT(f, params):
    return 2 * np.pi * f * time_to_coal(f, params) - phase_f(f, params)

def Psi(f, params, TTC, PHI_C):
    return 2 * np.pi * f * TTC - PHI_C - np.pi / 4 - PhiT(f, params) 

# Trapezoid method --------------------------------------------------------------------
    
def PhiT_cumul(f, params):
    return 2 * np.pi * f * time_to_coal_cumul(f, params) - phase_f_cumul(f, params)
    
def Psi_cumul(f, params, TTC, PHI_C):
    return 2 * np.pi * f * TTC - PHI_C - np.pi / 4 - PhiT_cumul(f, params) 

### $\cdot$ Amplitude and noise

In [99]:
def amplitude(f, params):
    '''Amplitude averaged over inclination angle.'''
    return np.sqrt(4 / 5) * h_0(f, params) / params.Binary_init.dist

In [100]:
def get_frequency_noise(psd, fs):
    
    delta_f = fs[0] - fs[1]
    sigma = np.sqrt(psd(fs)/(4 * delta_f))
    not_zero = (sigma != 0)
    sigma_red = sigma[not_zero]
    noise_re = np.random.normal(0, sigma_red)
    noise_co = np.random.normal(0, sigma_red)

    noise_red = (1/np.sqrt(2)) * (noise_re + 1j * noise_co)

    noise = np.zeros(len(sigma), dtype=complex)
    noise[not_zero] = noise_red

    return noise

### $\cdot$ Project the two waves and match

In [112]:
def calculate_match_unnormd_fft(
    params_h, fs, pad_low, pad_high, S_n=S_n
):
    """
    Inner product of waveforms, maximized over Phi_c by taking absolute value
    and t_c using the fast Fourier transform.
    """
    df = fs[0] - fs[1] # I have set out a reversed grid wr to the pydd version
    
    # define the waveform and match
    
    wf_h = amplitude(fs, params_h) * np.exp(1j * Psi_cumul(fs, params_h, TTC=0.0, PHI_C=0.0)) 
    wf_d = amplitude(fs, params_h) * np.exp(1j * Psi(fs, params_h, TTC=0.0, PHI_C=0.0)) 
    Sns = S_n(fs)

    # Use IFFT trick to maximize over t_c. Ref: Maggiore's book, eq. 7.171.
    integrand = 4 * wf_h.conj() * wf_d / Sns * df 
    integrand_padded = np.concatenate((pad_low, integrand, pad_high))
    
    # Maximize over time shift using IFFT trick
    unnormalized_match = np.abs(len(integrand_padded) * np.fft.ifft(integrand_padded)).max()
    
    # Now calculate the normalization terms (self-inner products)
    inner_product_hh = 4 * wf_h.conj() * wf_h / Sns * df
    inner_product_hh_padded = np.concatenate((pad_low, inner_product_hh, pad_high))
    norm_h = np.abs(len(inner_product_hh_padded) * np.fft.ifft(inner_product_hh_padded)).max()

    inner_product_dd = 4 * wf_d.conj() * wf_d / Sns * df
    inner_product_dd_padded = np.concatenate((pad_low, inner_product_dd, pad_high))
    norm_d = np.abs(len(inner_product_dd_padded) * np.fft.ifft(inner_product_dd_padded)).max()

    # Normalize the match
    normalized_match = unnormalized_match / np.sqrt(norm_h * norm_d)
    
    return normalized_match

In [114]:
def mismatch(params_h, fs, pad_low, pad_high, S_n=S_n):
    
    return 1 - calculate_match_unnormd_fft(params_h, fs, pad_low, pad_high, S_n=S_n)

# Test the matching theory

In [110]:
m1 = 1e5 * MSUN # kg
m2 = 10 * MSUN # kg
Mtot = m1 + m2
r_s = 2 * G * Mtot/ C**2 # Schwartzschild radius of m1
r0 = 3 * r_s
Mach = 100 
sigma0 = 1.5e10 / Mach**2
alpha = -1/2
q = m2/m1
M_chirp = m1**(3/5) * m2**(3/5) / Mtot**(1/5)

AD = myAccretionDisk(
    mach=Mach, 
    m1=m1,
    m2=m2,
    r0=r0, 
    dist=100e6 * PC,
    sigma0=sigma0, 
    alpha=alpha, 
    chirp_mass=(m1 * m2)**(3/5) / (m1 + m2)**(1/5))

r_isco = 6 * G * Mtot / C**2
f_isco = AD.Binary_init.frequency(r_isco)

In [104]:
FS = np.linspace(f_isco, 1e-3, 100000)
PAD_LOW, PAD_HIGH = get_match_pads(FS[::-1])  # padding for likelihood calculation

In [ ]:
# find mismatch

mis_match = mismatch(AD, FS, PAD_LOW, PAD_HIGH, S_n=S_n)

In [115]:
print('The mismatch is ', mismatch)

8.780571888467037e-07

THEY ARE MATCHING!